# Importing all the packages

In [1]:
from argparse import Namespace
from collections import Counter
import json
import os
import re
import string
import itertools
import numpy as np
import pandas as pd
import torch
import tqdm
from torch.utils.data.sampler import SubsetRandomSampler
CUDA = torch.cuda.is_available()
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook

In [ ]:
torch.manual_seed(71)

In [ ]:
CUDA

True

In [ ]:
CONTEXT_SIZE = 4
EMBEDDING_DIM = 50

# Reading the file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/My Drive")
!ls

 1030_2.vb	    Resume.docx			   'Untitled document.gdoc'
'Colab Notebooks'  'Rishav Resume.pdf'
'Google Buzz'	    tripadvisor_hotel_reviews.csv


In [ ]:
df = pd.read_csv('tripadvisor_hotel_reviews.csv')

In [ ]:
df.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20491 entries, 0 to 20490
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  20491 non-null  object
 1   Rating  20491 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 320.3+ KB


In [ ]:
del df['Rating']

# Clean Dataset

In [ ]:
df['Review'] = df['Review'].apply(lambda x: re.sub('[!@*#/$:).;,?&(\{\}\[\]\d+]', '', x.lower()))
df['Review'] = df['Review'].apply(lambda x: re.sub('  ', ' ', x))

In [ ]:
df.head()

,Review
0,nice hotel expensive parking got good deal sta...
1,ok nothing special charge diamond member hilto...
2,nice rooms not experience hotel monaco seattle...
3,unique great stay wonderful time hotel monaco ...
4,great stay great stay went seahawk game awesom...


# Combine all the items together

In [ ]:
all_reviews = df['Review'].tolist()

In [ ]:
len(all_reviews), df.shape

(20491, (20491, 1))

# Method to join all the records in the list

In [ ]:
def joinList(reviews):
    raw_data = " ".join(all_reviews)
    return raw_data

# Reading the Scifi file

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
os.chdir("/content/drive/My Drive")
!ls

'Colab Notebooks'
 DSC_1060.JPG
 DSC_1756.JPG
 DSC_1759.JPG
 DSC_1891.JPG
 embeddings_scifi1_40epoc_60emb.npy
 ex01_lc.ipynb
 ex01_mlp.ipynb
'Ex 2-Scifi.ipynb'
 master.csv
'MS OFFICE ACTIVATOR.rar'
 MyResume.doc
 MyResume.doc.gdoc
 Resume
'Resume (1).docx'
 Resume.docx
 Road-Rash_Win_EN_RIP-Version.zip
 scifi.txt
 Statement_1535772563820.pdf
 tanu
 tripadvisor_hotel_reviews.gsheet
'tripadvisor_hotel_reviews - tripadvisor_hotel_reviews.csv'
'Untitled document.gdoc'
'Veronica Chelea Sustainable Tourism Development (1).gslides'


In [7]:
with open('scifi.txt', 'r') as text:
    textfile = text.read()

# Clean Dataset

In [8]:
textfile = re.sub(r'\s+', ' ',textfile)
textfile = re.sub('[-!@*#/$:).;,?&(\{\}\[\]\d+]', '', textfile)
textfile = re.sub(r'\s+', ' ',textfile)
textfile = re.sub(r'['+string.punctuation+']', '',textfile)
textfile = re.sub('  ','',textfile)

# Combine all the items together

In [9]:
textfile_list = textfile.split()
vocab = set(textfile_list)
word2index = {w:i for i,w in enumerate(vocab)}
index2word = {i:w for i,w in enumerate(vocab)}
    
textfile_list = textfile_list[:2000000]
vocab = set(textfile_list)
word2index = {w:i for i,w in enumerate(vocab)}
index2word = {i:w for i,w in enumerate(vocab)}


## Class for building the Vocabulary

In [ ]:
class Vocabulary(object):
    def __init__(self, token_to_idx=None):
        if token_to_idx is None:
            token_to_idx = {}
            
        self._token_to_idx = token_to_idx

        self._idx_to_token = {idx: token 
                              for token, idx in self._token_to_idx.items()}
        
    def add_token(self, token):
        index = len(self._token_to_idx)
        self._token_to_idx[token] = index
        self._idx_to_token[index] = token
        return index
    
    def getidx(self):
        return self._token_to_idx

    def lookup_token(self, token):
        return self._token_to_idx[token]

    def lookup_index(self, index):
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]


In [ ]:
# test = Vocabulary()
# raw_text = """We Chennai about to study the idea of a computational process.
# Computational processes are Chennai beings that inhabit computers.
# As they evolve, Chennai manimpulate other abstract things called data.
# The evolution of a process is Chennai by a pattern of rules
# called a program. People create programs to direct processes. In effect,
# we conjure the spirits of the computer with our spells.""".split()
 

In [ ]:
# for word in raw_text:
#     test.add_token(word) 
    

In [ ]:
# print(test.getidx())

## Class for building the Vectorizer

In [ ]:
class ReviewVectorizer(object):
    def __init__(self, review_vocab=None):
        self.review_vocab = review_vocab

    def vectorize(self):
        data = []
        for i in range(2, len(self.review_vocab)-2):
            context = [self.review_vocab[i-2], self.review_vocab[i-1], self.review_vocab[i+1], self.review_vocab[i+2]]
            target = self.review_vocab[i]
            data.append((context,target))
        #print("data -> ",data[:5])
        return data
    

        

    

In [ ]:
# raw_text = """We are about to study the idea of a computational process.
# Computational processes are abstract beings that inhabit computers.
# As they evolve, processes manimpulate other abstract things called data.
# The evolution of a process is directed by a pattern of rules
# called a program. People create programs to direct processes. In effect,
# we conjure the spirits of the computer with our spells.""".split() 

In [ ]:
# reviewVec = ReviewVectorizer(raw_text) 

In [ ]:
# dataret = reviewVec.vectorize()

In [ ]:
# print(dataret)

In [ ]:
# new_df = pd.DataFrame(dataret, columns=["context_review","target_review"])

In [ ]:
# new_df

In [ ]:
# def get_index(x):
#     return (list(test.lookup_token(token) for token in x))
# new_df['context_review'] = new_df['context_review'].apply(get_index)
# new_df['target_review'] = new_df['target_review'].apply(test.lookup_token)


In [ ]:
# new_df

# Creating Dataset Class

In [ ]:
from sklearn.model_selection import train_test_split

class ReviewDataset(Dataset):
    def __init__(self, X, y, scale_X=True):
        if not torch.is_tensor(X):
            if CUDA:
                self.X = torch.tensor(X).cuda()
            else:
                self.X = torch.tensor(X)
        if not torch.is_tensor(y):
            if CUDA:
                self.y = torch.tensor(y).cuda()
            else:
                self.y = torch.tensor(y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

    def __len__(self):
        return len(self.X)
    
    
def generate_batches(dataset, batch_size,
                     drop_last=False, device="cpu"):
    train, test = train_test_split(list(range(dataset.X.shape[0])), test_size=.3)
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            sampler=SubsetRandomSampler(train),num_workers=0)

    return dataloader

In [ ]:
# dataloader = DataLoader(dataset=dataset, batch_size=32,
#                             sampler=SubsetRandomSampler(train))

# Creating Model Class

In [ ]:
class CBOW2(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW2,self).__init__()
        self.embeddings = nn.Embedding(vocab_size,embedding_dim)
        if CUDA:
            self.embeddings = self.embeddings.cuda()
        self.layer1 = nn.Linear(embedding_dim, 128)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(128, vocab_size)
    
    def forward(self, data_input):
#         print("data_input shape",data_input.shape)
        embeddings = self.embeddings(data_input.long())
#         print("embedding shape", embeddings.shape)
        output_layer1 = self.layer1(embeddings)
#         print("output layer 1 shape", output_layer1.shape)
        output_layer1 = torch.mean(output_layer1, dim=1)
#         print("Output layer 1 after mean shape", output_layer1.shape)
        output_layer1_relu = self.relu(output_layer1)
#         print("output layer 1 shape relu", output_layer1.shape)
        output_layer2 = self.layer2(output_layer1)
#         print("output layer 2 shape", output_layer2.shape)
        log_softmax = F.log_softmax(output_layer2)
        return log_softmax

    
    def write_embedding_to_file(self,filename):
      for i in self.embeddings.parameters():
        weights = i.data.numpy()
      np.save(filename,weights)

# Creating the runner class for Tripadvisor dataset to process further

In [ ]:
def runner():
    all_reviews = df['Review'].tolist()
   
    corpus_df = [x.split() for x in all_reviews]
    vocab = list(set(itertools.chain.from_iterable(corpus_df)))
    corpus = list(itertools.chain.from_iterable(corpus_df))
    
    word2index = {w:i for i,w in enumerate(vocab)}
    index2word = {i:w for i,w in enumerate(vocab)}


    review_vectorizer = ReviewVectorizer(corpus)


    context_target_pair = review_vectorizer.vectorize()
    

    # print(context_target_pair)

    review_new_df = pd.DataFrame(context_target_pair, columns=["context_review","target_review"])
    # print(review_new_df)
    
    def get_index(x):
        return (list(word2index[token] for token in x))

    review_new_df['context_review'] = review_new_df['context_review'].apply(get_index)
    review_new_df['target_review'] = review_new_df['target_review'].apply(lambda x: word2index[x])
    
    # print(review_new_df)
    
    X, y = np.array(review_new_df['context_review'].tolist()), np.array(review_new_df['target_review'].tolist())
    dataset = ReviewDataset(X,y)
    
    # print("dataset->",dataset)
    
    dataloader = generate_batches(dataset=dataset,batch_size=64)
    
    return dataloader,len(vocab)
    
   

In [ ]:
dataloader,vocab_length = runner()

In [ ]:
iter(dataloader).__next__()

[tensor([[43737, 44832, 32328, 68666],
         [39101, 31144, 49980, 49980],
         [50396, 67379, 43058,  2209],
         [33126, 34818, 42311, 47976],
         [42786, 20104, 31212, 51362],
         [29889, 74203, 13981,  7970],
         [46148, 70191, 22988, 17418],
         [58465, 71840, 63830, 51362],
         [76987, 34575, 43048, 12648],
         [62805, 22417, 18439, 13454],
         [ 8619, 72023, 22311, 65951],
         [59494, 38840, 56613, 61753],
         [30253, 24107, 74093, 64208],
         [32328, 24785, 32328, 43909],
         [32328, 36462, 15152, 43316],
         [32783, 53907, 43048,  3722],
         [58465, 15237, 30253, 63755],
         [26174, 17657, 32423,  8506],
         [56244, 76439, 57220, 56244],
         [65662, 54185, 68666, 65966],
         [57010, 29847, 45520, 49894],
         [57010, 62805, 34687, 20979],
         [31272, 65834, 32086, 48398],
         [46871, 12366, 26622, 51741],
         [26153, 24584,  7637, 38063],
         [56452, 21052, 2

In [ ]:
# for index,(context, target) in enumerate(dataloader):

#         # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
#         # into integer indices and wrap them in tensors)
#         context_tensor = Variable(context)
#         target_tensor = Variable(target)
        
#         print(context_tensor.shape)
#         print(target_tensor.shape)

In [ ]:
print(vocab_length)
np.sqrt(4)

78784


2.0

In [ ]:
losses = []
loss_function = torch.nn.CrossEntropyLoss()
model = CBOW2(vocab_length, EMBEDDING_DIM)
if CUDA:
    model = model.cuda()

optimizer = optim.Adagrad(model.parameters(), lr=0.001)

for epoch in tqdm.tqdm(range(50)):
    total_loss = 0
    for index,(context, target) in enumerate(dataloader):

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_tensor = Variable(context)
        target_tensor = Variable(target)
        
        preds = model(context_tensor)
        target = target.type(torch.LongTensor)
        
        loss = loss_function(preds, target_tensor)


        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
#         log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
#         loss = loss_function(log_probs, torch.tensor([ret[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
    
    mbl = np.mean(np.sqrt(losses)).round(3)
    if epoch % 5 == 0:
        print("Epoch [{}/{}], Batch loss: {}".format(epoch, 50, mbl))  #

  2%|▏         | 1/50 [02:53<2:21:33, 173.33s/it]

Epoch [0/50], Batch loss: 437.357


 12%|█▏        | 6/50 [17:17<2:06:33, 172.58s/it]

Epoch [5/50], Batch loss: 424.493


 22%|██▏       | 11/50 [31:36<1:51:45, 171.95s/it]

...

 82%|████████▏ | 41/50 [1:57:37<25:45, 171.75s/it]

Epoch [40/50], Batch loss: 417.523


 92%|█████████▏| 46/50 [2:11:54<11:26, 171.55s/it]

Epoch [45/50], Batch loss: 417.014


 94%|█████████▍| 47/50 [2:14:46<08:34, 171.47s/it]

In [ ]:
model.cpu().write_embedding_to_file('embeddings_table_40epoc_60emb.npy')

# Creating the runner class for Scifi dataset to process 

In [27]:
def runner():
   
    vocab = set(textfile_list)
    word2index = {w:i for i,w in enumerate(vocab)}
    index2word = {i:w for i,w in enumerate(vocab)}
    


    review_vectorizer = ReviewVectorizer(textfile_list)


    context_target_pair = review_vectorizer.vectorize()
    

    print(context_target_pair)

    review_new_df = pd.DataFrame(context_target_pair, columns=["context_review","target_review"])
    print(review_new_df)
    
    def get_index(x):
        return (list(word2index[token] for token in x))

    review_new_df['context_review'] = review_new_df['context_review'].apply(get_index)
    review_new_df['target_review'] = review_new_df['target_review'].apply(lambda x: word2index[x])
    
    print(review_new_df)
    
    X, y = np.array(review_new_df['context_review'].tolist()), np.array(review_new_df['target_review'].tolist())
    dataset = ReviewDataset(X,y)
    
    print("dataset->",dataset)
    
    dataloader = generate_batches(dataset=dataset,batch_size=64)
    
    return dataloader,len(vocab)
    
   

In [28]:
dataloader,vocab_length = runner()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



                              context_review target_review
0                     [MARCH, All, New, and]       Stories
1              [All, Stories, and, Complete]           New
2        [Stories, New, Complete, Publisher]           and
3              [New, and, Publisher, Editor]      Complete
4                [and, Complete, Editor, IF]     Publisher
...                                      ...           ...
1999991         [perversity, of, man, whose]             a
1999992                 [of, a, whose, mind]           man
1999993                  [a, man, mind, and]         whose
1999994             [man, whose, and, heart]          mind
1999995            [whose, mind, heart, are]           and

[1999996 rows x 2 columns]
                       context_review  target_review
0        [37008, 60454, 39271, 31197]          24068
1        [60454, 24068, 31197, 22903]          39271
2          [24068, 39271, 22903, 872]          31197
3          [39271, 31197, 872, 44926]          22903

In [29]:
iter(dataloader).__next__()

[tensor([[57207, 24697, 44153, 31197],
         [44332, 10544,  9721, 52127],
         [16148,  7486, 50840, 59899],
         [56435, 43178, 45580, 38811],
         [52333, 12177, 13440,  3094],
         [51144, 31209, 24697,  7486],
         [12516, 50840, 42201, 30961],
         [43084, 16275, 15876, 21511],
         [61655, 36829, 57207, 31197],
         [53941, 63463, 37778,  7486],
         [17792, 31539, 43885, 26711],
         [63484, 24697,   657, 19735],
         [32683,  3094,  7693,  3132],
         [55709, 49128,  3094,  6347],
         [14550, 56411, 59899, 38987],
         [19728,  9929, 54835, 12046],
         [52680,  2670, 62558, 42167],
         [26711, 42389, 11562, 59899],
         [65619, 37693, 48405, 44862],
         [51144,  1435, 18105, 48405],
         [ 3116, 33050, 24714, 23051],
         [60371, 34310, 21901,  7332],
         [61655, 56060, 26711, 51144],
         [25838, 15522, 42434,  9975],
         [14241, 45753, 51144, 27945],
         [13440,  7990, 2

In [30]:
# for index,(context, target) in enumerate(dataloader):

#         # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
#         # into integer indices and wrap them in tensors)
#         context_tensor = Variable(context)
#         target_tensor = Variable(target)
        
#         print(context_tensor.shape)
#         print(target_tensor.shape)

In [31]:
print(vocab_length)

66521


In [33]:
losses = []
loss_function = torch.nn.CrossEntropyLoss()
model = CBOW2(vocab_length, EMBEDDING_DIM)
if CUDA:
    model = model.cuda()

optimizer = optim.Adagrad(model.parameters(), lr=0.001)

for epoch in tqdm.tqdm(range(50)):
    total_loss = 0
    for index,(context, target) in enumerate(dataloader):

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_tensor = Variable(context)
        target_tensor = Variable(target)
        
        preds = model(context_tensor)
        target = target.type(torch.LongTensor)
        
        loss = loss_function(preds, target_tensor)


        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
#         log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
#         loss = loss_function(log_probs, torch.tensor([ret[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)

    mbl = np.mean(np.sqrt(losses)).round(3)
    if epoch % 5 == 0:
        print("Epoch [{}/{}], Batch loss: {}".format(epoch
                                                     , 50, mbl))  #

  2%|▏         | 1/50 [02:25<1:58:39, 145.30s/it]

Epoch [0/50], Batch loss: 418.469


 12%|█▏        | 6/50 [14:26<1:45:58, 144.52s/it]

Epoch [5/50], Batch loss: 402.465


 22%|██▏       | 11/50 [26:32<1:34:23, 145.21s/it]

...

 82%|████████▏ | 41/50 [1:39:21<21:52, 145.78s/it]

Epoch [40/50], Batch loss: 393.801


 92%|█████████▏| 46/50 [1:51:28<09:40, 145.15s/it]

Epoch [45/50], Batch loss: 393.275


100%|██████████| 50/50 [2:01:04<00:00, 145.29s/it]


In [4]:
verb = ['was','had',  'have']
noun = ['man', 'eyes', 'people']
adjective = ['first', 'good', 'different']

In [100]:
model_trip = model

In [111]:
model.embedding.weight.cpu()

tensor([[ 2.3494,  0.2627,  0.0063,  ..., -1.5024, -0.7096,  1.7995],
        [ 1.6593, -1.3654, -4.1932,  ...,  1.1848, -2.0572,  3.5361],
        [ 0.1847, -1.8835, -0.1488,  ..., -0.2514,  0.8871, -0.0710],
        ...,
        [ 0.3431,  1.1005,  0.1094,  ..., -1.1764,  0.2043,  2.0469],
        [ 1.8896,  0.3658,  1.8923,  ..., -1.2124, -1.1381,  0.2541],
        [-0.1498, -1.2524,  1.8340,  ..., -2.4046, -2.9674, -2.4239]],
       grad_fn=<CopyBackwards>)

In [114]:
type(model_trip.embedding.weight.cpu().detach().numpy())

numpy.ndarray

In [115]:
word_emb_dict = dict(zip(vocab, model_trip.embedding.weight.cpu().detach().numpy()))

In [116]:
def get_closest_word(word, topn = 5):
    word_distance = []
    emb = word_emb_dict
    pdist = nn.PairwiseDistance()
    i = word2index[word]
    v_i = torch.tensor(emb[word]).view(1,-1)
    for j in range(len(vocab)):
        if j != i:
            lookup_tensor_word = index2word[j]
            v_j = torch.tensor(emb[lookup_tensor_word]).view(1,-1)
            distance = pdist(v_i, v_j)
            distance = distance.numpy()
            word_distance.append( (lookup_tensor_word, float(distance)) )
    word_distance.sort(key=lambda x: x[1]) 
    return word_distance[:topn]

In [117]:
from collections import Counter
word_counts = Counter(corpus)
word_counts_df = pd.DataFrame.from_dict(word_counts, orient='index').reset_index()
word_counts_df.columns = ['words', 'counts']
word_counts_df.sort_values(by='counts', ascending=False, inplace=True)

In [119]:
pd.options.display.max_rows = 1000

In [126]:
common_words = ['the', 'a']

In [128]:
for word in common_words:
  print("Most common words in the corpus -->", word, " and its nearest neighbours are:")
  print(get_closest_word(word, topn=3))

Most common words in the corpus --> the  and its nearest neighbours are:
[('courteouswe', 14.91102409362793), ('amusic', 15.502732276916504), ('crazymoney', 15.514968872070312)]
Most common words in the corpus --> a  and its nearest neighbours are:
[('fiddled', 13.259047508239746), ('terrific', 13.411040306091309), ('cottagesbungalow', 13.41502571105957)]


In [124]:
word_counts_df[word_counts_df.words == 'a']

,words,counts
8170,a,73


In [91]:
from collections import Counter
word_counts = Counter(corpus)
word_counts_df = pd.DataFrame.from_dict(word_counts, orient='index').reset_index()
word_counts_df.columns = ['words', 'counts']
word_counts_df.sort_values(by='counts', ascending=False, inplace=True)

In [92]:
pd.options.display.max_rows = 1000

In [87]:
verb = ['was','had',  'have']
noun = ['man', 'eyes', 'people']
adjective = ['first', 'good', 'different']

In [93]:
for word in verb:
  print("Most common verbs in the corpus -->", word, " and its nearest neighbours are:")
  print(get_closest_word(word))

Most common verbs in the corpus - was  and its nearest neighbours are:
[('nextincommands', 14.108367919921875), ('wasnt', 14.500380516052246), ('alish', 14.795513153076172), ('tacitly', 14.84989070892334), ('cancellation', 14.955382347106934)]
Most common verbs in the corpus - had  and its nearest neighbours are:
[('bfen', 14.994134902954102), ('dornots', 15.030157089233398), ('atlanta', 15.20980167388916), ('conveying', 15.240338325500488), ('hadnt', 15.268654823303223)]
Most common verbs in the corpus - have  and its nearest neighbours are:
[('lascdio', 14.696041107177734), ('chosing', 14.704346656799316), ('foreknowledge', 14.778069496154785), ('goofs', 15.003154754638672), ('lanargon', 15.173529624938965)]


In [94]:
for word in noun:
  print("Most common noun in the corpus -->", word, " and its nearest neighbours are:")
  print(get_closest_word(word))

Most common noun in the corpus --> man  and its nearest neighbours are:
[('person', 14.04078197479248), ('cwtrol', 14.744359970092773), ('auid', 14.977381706237793), ('timetravelers', 15.193326950073242), ('gorgeously', 15.277854919433594)]
Most common noun in the corpus --> eyes  and its nearest neighbours are:
[('parrotlike', 17.926855087280273), ('bevatron', 18.14840316772461), ('thoughts', 18.29469871520996), ('aunties', 18.540016174316406), ('itf', 18.575946807861328)]
Most common noun in the corpus --> people  and its nearest neighbours are:
[('compartnienls', 13.266700744628906), ('men', 13.309320449829102), ('uninteresting', 13.346628189086914), ('dulling', 13.83996295928955), ('resurrect', 13.864710807800293)]


In [95]:
for word in adjective:
  print("Most common adjective in the corpus -->", word, " and its nearest neighbours are:")
  print(get_closest_word(word))

Most common adjective in the corpus --> first  and its nearest neighbours are:
[('propagandist', 14.926827430725098), ('whereever', 15.226884841918945), ('coiils', 15.30133056640625), ('seventeento', 15.305699348449707), ('saltpan', 15.426239013671875)]
Most common adjective in the corpus --> good  and its nearest neighbours are:
[('thighlength', 14.115311622619629), ('betuned', 14.56428050994873), ('pharaoh', 14.575023651123047), ('erde', 14.591188430786133), ('isoi', 14.591279983520508)]
Most common adjective in the corpus --> different  and its nearest neighbours are:
[('coptered', 15.005867004394531), ('lashings', 15.01534366607666), ('statism', 15.02927303314209), ('bfen', 15.057822227478027), ('blindwindows', 15.09548282623291)]


In [99]:
common_words = ['the', 'a']
for word in common_words:
  print("Most common words in the corpus -->", word, " and its nearest neighbours are:")
  print(get_closest_word(word, 3))

Most common words in the corpus --> the  and its nearest neighbours are:
[('muggier', 11.156322479248047), ('anticomputerites', 11.638870239257812), ('wellfurnished', 11.651493072509766)]
Most common words in the corpus --> a  and its nearest neighbours are:
[('randalls', 14.078964233398438), ('peasized', 14.20406436920166), ('flavoring', 14.23133373260498)]
